# CME538 - Introduction to Data Science
## Assignment 8 - Data Science Life Cycle

### Learning Objectives
After completing this assignment, you should be comfortable:

- Feature engineering
- Using sklearn to build simple and more complex linear models
- Building a data pipeline using pandas
- Identifying informative variables through EDA
- Feature engineering with categorical variables
- Classification using logistic regression
- Classification metrics

### Marking Breakdown

Question | Points
--- | ---
Question 1a | 1
Question 1b | 1
Question 1c | 1
Question 1d | 1
Question 2a | 1
Question 2b | 1
Question 2c | 1
Question 3a | 1
Question 3b | 1
Question 3c | 1
Question 3d | 1
Question 4a | 1
Question 4b | 1
Question 4c | 1
Question 4d | 1
Question 4e | 1
Question 4f | 1
Total | 17

One of the following marks below will be added to the **Total** above.

### Code Quality

| Rank | Points | Description |
| :-- | :-- | :-- |
| Youngling | 1 | Code is unorganized, variables names are not descriptive, redundant, memory-intensive, computationally-intensive, uncommented, error-prone, difficult to understand. |
| Padawan | 2 | Code is organized, variables names are descriptive, satisfactory utilization of memory and computational resources, satisfactory commenting, readable. |
| Jedi | 3 | Code is organized, easy to understand, efficient, clean, a pleasure to read. #cleancode |

## Setup Notebook

In [ ]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# Overview

In this assignment, you will use all the tools and knowledge that you've learned in class to explore and model New York Taxi data. You will create a regression model that predicts the travel time of a taxi ride and you will also create a binary classifier to predict whether or not someone will tip at the end of their ride. This assignment is meant to guide you through the complete Data Science Life Cycle. 

# The Data

Features of all [yellow taxi](https://en.wikipedia.org/wiki/Taxicabs_of_New_York_City) trips in January 2016 are published by the [NYC Taxi and Limosine Commission](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

The full dataset is over 1 GB, so we've placed a simple random sample of the data into `taxi.csv`.

Columns of the `taxi.csv` include:
- `pickup_datetime`: date and time when the meter was engaged
- `dropoff_datetime`: date and time when the meter was disengaged
- `pickup_lon`: the longitude where the meter was engaged
- `pickup_lat`: the latitude where the meter was engaged
- `dropoff_lon`: the longitude where the meter was disengaged
- `dropoff_lat`: the latitude where the meter was disengaged
- `passengers`: the number of passengers in the vehicle (driver entered value)
- `distance`: trip distance in miles
- `payment_method`: 1=Credit card, 2=Cash, 3=No charge, 4=Dispute
- `surcharge`: Improvement surcharge
- `tax`: State and municipal taxes
- `fare`: the time-and-distance fare calculated by the meter
- `tip`: the amount of credit card tips, cash tips are not included

# 1. Data Selection and Cleaning

In this part, you will limit the data to trips that began and ended on Manhattan Island ([map](https://www.google.com/maps/place/Manhattan,+New+York,+NY/@40.7590402,-74.0394431,12z/data=!3m1!4b1!4m5!3m4!1s0x89c2588f046ee661:0xa0b3281fcecc08c!8m2!3d40.7830603!4d-73.9712488)). 

## Question 1a
Import `taxi.csv` as a DataFrame and assign it to a variable called `all_taxi`. 

Only include trips that have **both** pick-up and drop-off locations within the boundaries of New York City:

- Longitude is between -74.03 and -73.75 (inclusive of both boundaries)
- Latitude is between 40.6 and 40.88 (inclusive of both boundaries)

In [ ]:
# Write your code here
all_taxi = ...

# View DataFrame
all_taxi.head()

## Question 1b
Create a plot of pickup locations using Folium and the `HeatMap` function. The `HeatMap` will show the density of pickup locations in different areas of the city, where red areas have relatively more pickups. As you can see, most of the pickups are in Manhattan in addition to two hot spots corresponding to airports. Your plot should look like this (radius = 10).
<br>
<img src="images/q1b.png" alt="drawing" width="500"/>
<br> 

In [ ]:
!pip install folium

In [ ]:
import folium
from folium import Circle
from folium.plugins import HeatMap

In [ ]:
# Create a map of New York
map_1 = folium.Map(location=[40.7484, -73.9857], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Write your code here
...

# Display map
map_1

## Question 1c
We'll be trying to estimate the duration of a taxi ride and therefore, we will need to compute the duration of each ride. Create a new column in `all_taxi` called `'duration'` and assign to it the length of the taxi ride in seconds. The datatype for `'duration'` should be `int`. 

In [ ]:
# Convert to datetime and localize to EST
all_taxi['pickup_datetime'] = pd.to_datetime(all_taxi['pickup_datetime'], errors='coerce')
all_taxi['dropoff_datetime'] = pd.to_datetime(all_taxi['dropoff_datetime'], errors='coerce')
all_taxi['pickup_datetime'] = all_taxi['pickup_datetime'].dt.tz_localize(tz='EST')
all_taxi['dropoff_datetime'] = all_taxi['dropoff_datetime'].dt.tz_localize(tz='EST')

# Write your code here
all_taxi['duration'] = ...

# View DataFrame
all_taxi.head()

## Question 1d
Create a DataFrame called `clean_taxi` that only includes trips with a positive passenger count, a positive distance, a duration of at least 1 minute and at most 1 hour, and an average speed of at most 100 miles per hour. Inequalities should not be strict (e.g., `<=` instead of `<`) unless comparing to 0. You will need to first create a new column in `all_taxi` called `'speed'` and assign to it the average speed in miles per hour.

In [ ]:
# Write your code here
all_taxi['speed'] = ...
clean_taxi = ...

# View DataFrame
clean_taxi.head()

Next, we want to collect trips from `clean_taxi` that start and end within the boundaries of Manhattan Island.

We're in luck because `GeoPandas` has a builtin dataset for the boundaries of New York Boroughs.

In [ ]:
path = gpd.datasets.get_path('nybb')
boroughs = gpd.read_file(path)
boroughs.set_index('BoroCode', inplace=True)
boroughs.sort_index(inplace=True)
boroughs.to_crs(epsg=4326, inplace=True)
boroughs.head()

Let's plot these quickly.

In [ ]:
ax = boroughs.plot(figsize=(8, 8), column='BoroName', categorical=True, 
                   cmap='Accent', linewidth=.6, edgecolor='0.2', alpha=0.5,
                   legend=True, legend_kwds={'loc': 2,'fontsize':16, 
                                             'frameon':False})

Let's create a new DataFrame called `manhattan_taxi` that only includes trips from `clean_taxi` that start and end within the boundaries of Manhattan Island.

In [ ]:
!pip install shapely

In [ ]:
from shapely.geometry import MultiPoint

First we have to create a `MultiPoint` geometry for each trip (pickup and dropoff).

In [ ]:
clean_taxi['geometry'] = clean_taxi.apply(lambda row: MultiPoint([(row['pickup_lon'], row['pickup_lat']), 
                                                                  (row['dropoff_lon'], row['dropoff_lat'])]), axis=1)
clean_taxi = gpd.GeoDataFrame(clean_taxi)
clean_taxi.head()

Next, for each trip we need to check if it started and ended within Manhattan.

In [ ]:
manhattan_taxi = clean_taxi[clean_taxi.within(boroughs[boroughs['BoroName'] == 'Manhattan'].iloc[0].geometry)]

Let's see if our filtering worked.

In [ ]:
ax = boroughs.plot(figsize=(8, 8), column='BoroName', categorical=True, 
                   cmap='Accent', linewidth=.6, edgecolor='0.2', alpha=0.5,
                   legend=True, legend_kwds={'loc': 2,'fontsize':16, 
                                             'frameon':False})
manhattan_taxi.plot(ax=ax, markersize=0.00001, color='r');

It worked! That filtering operation takes quite a bit of time, so let's save a `.csv` and create a checkpoint.

In [ ]:
manhattan_taxi.to_csv('manhattan_taxi.csv', index=False)

## Question 1e (ungraded)
In the following cell, print a summary of the data selection and cleaning you performed. For example, you should print something like: "Of the original 1000 trips, 21 anomolous trips (2.1%) were removed through data cleaning, and then the 600 trips within Manhattan were selected for further analysis." (Note that the numbers in this example are not accurate.)

_Type your answer here, replacing this text._

# Predicting Trip Duration
In this section, you will develop a machine learning model to predict the duration of taxi trips in New York.

# 2. Exploratory Data Analysis
In this part, you'll choose which days to include as training data in your regression model. 

Your goal is to develop a general model that could potentially be used for future taxi rides. There is no guarantee that future distributions will resemble observed distributions, but some effort to limit training data to typical examples can help ensure that the training data are representative of future observations.

January 2016 had some atypical days. New Years Day (January 1) fell on a Friday. MLK Day was on Monday, January 18. A [historic blizzard](https://en.wikipedia.org/wiki/January_2016_United_States_blizzard) passed through New York that month. Using this dataset to train a general regression model for taxi trip times must account for these unusual phenomena, and one way to account for them is to remove atypical days from the training data.

## Question 2a
Add a new column named `'date'` to `manhattan_taxi` that contains the date (but not the time) of pickup.

In [ ]:
# Write your code here
manhattan_taxi['date'] = ...

# View DataFrame
manhattan_taxi.head()

## Question 2b
Create a data visualization that allows you to identify which dates were affected by the historic blizzard of January 2016. Make sure that the visualization type is appropriate for the visualized data.

Hint: How do you expect taxi usage to differ on blizzard days?

In [ ]:
# Write your code here
...

## Question 2c
We have generated a list of dates that should have a fairly typical distribution of taxi rides, which excludes holidays and blizzards.

In [ ]:
import re
import calendar

print('Typical dates:\n')
pat = '  [1-3]|18 | 23| 24|25 |26 '
print(re.sub(pat, '   ', calendar.month(2016, 1)))

In [ ]:
atypical = [1, 2, 3, 18, 23, 24, 25, 26]
typical_dates = [n for n in range(1, 32) if n not in atypical]
print(typical_dates)

Create a new DataFrame that only contains `typical_dates` and assign it to a new variable called `final_taxi`.

In [ ]:
# Write your code here
final_taxi = ...

# View DataFrame
final_taxi.head()

You are welcome to perform more exploratory data analysis, but your work will not be scored. Here's a blank cell to use if you wish. In practice, further exploration would be warranted at this point, but we won't require that of you.

In [ ]:
# Write your code here

# 3. Feature Engineering

In this section, you'll create a feature set for your linear regression model. You decide to predict trip duration from the following inputs: start location, end location, trip distance, time of day, and day of the week (*Monday, Tuesday, etc.*). 

You will ensure that the process of transforming observations into a feature set is expressed as a Python function called `create_features`, so that it's easy to make predictions for different samples in later parts of the assignment.

Because you are going to look at the data in detail in order to define features, it's best to split the data into training, validation and test sets now, then only inspect the training set. Remember what we learned in Lecture 21 about a form a data leakage called train/test contamination. Do not touch your test dataset until your final model is trained.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_taxi, train_size=0.7, 
                               test_size=0.3, random_state=0)
val, test = train_test_split(test, train_size=0.5,
                             test_size=0.5, random_state=0)
print('Train:', train.shape, 'Val:', val.shape, 'Test:', test.shape)

## Question 3a
Create a box plot that compares the distributions of taxi trip durations for each day using `train` only. Individual dates should appear on the horizontal axis, and duration values should appear on the vertical axis. Your plot should look like the following.

<br>
<img src="images/q3a.png" alt="drawing" width="500"/>
<br> 

Hint: Use sns.boxplot

In [ ]:
# Write your code here
...

## Question 3b
Add the following features to `train`, `val` and `test`. 

- `hour`: The integer hour of the pickup time. E.g., a 3:45pm taxi ride would have `15` as the hour. A 12:20am ride would have `0`.
- `day`: The day of the week with Monday=0, Sunday=6.
- `weekend`: 1 if and only if the `day` is Saturday or Sunday.
- `period`: 1 for early morning (12am-6am), 2 for daytime (6am-6pm), and 3 for night (6pm-12pm). Hint: np.digitize()

Because we have to do this for `train`, `val` and `test`, create a function called `add_features`.

In [ ]:
def add_features(df):
    """Augment a dataframe df with additional features."""
    df_temp = df.copy()
    
    # Write your code here
    df_temp.loc[:, 'hour'] = ...
    df_temp.loc[:, 'day'] = ...
    df_temp.loc[:, 'weekend'] = ...
    df_temp.loc[:, 'period'] = ...
    
    return df_temp

# Add features to train , val and test
train = add_features(train)
val = add_features(val)
test = add_features(test)

# View train DataFrame
train.head()

## Question 3c
Use `sns.distplot` and `train` to create an overlaid histogram comparing the distribution of average speeds for taxi rides that start in the early morning (12am-6am), day (6am-6pm; 12 hours), and night (6pm-12am; 6 hours). Your plot should look like this.
<br>
<img src="images/q3c.png" alt="drawing" width="500"/>
<br> 

In [ ]:
# Write your code here
...

Manhattan can roughly be divided into Lower, Midtown, and Upper regions. Instead of studying a map, let's approximate by finding the first principal component of the pick-up location (latitude and longitude). Before doing that, let's once again take a look at a scatterplot of trips in Manhattan.

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(manhattan_taxi['pickup_lon'], 
            manhattan_taxi['pickup_lat'], s=0.1, alpha=0.2)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pickup locations');

Add a `region` column to `train`, `val` and `test` that categorizes each pick-up location as 0, 1, or 2 based on the value of each point's first principal component, such that an equal number of points fall into each region. 

Read the documentation of [`pd.qcut`](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.qcut.html), which categorizes points in a distribution into equal-frequency bins.

In [ ]:
from sklearn.decomposition import PCA

First, let's calculate the first principal component (PCA1) using `'pickup_lon'` and `'pickup_lat'` from `train`.

In [ ]:
pca = PCA(n_components=1)
pca.fit(train[['pickup_lon', 'pickup_lat']])

In [ ]:
print('Principal component 1 explains {:.2f} % of the variance in "pickup_lon" and "pickup_lat".'.format(pca.explained_variance_ratio_[0] * 100))

Next, we need to transform `['pickup_lon', 'pickup_lat']` into principal component 1.

In [ ]:
pc1 = pca.transform(train[['pickup_lon', 'pickup_lat']])
print('Original shape: ', train[['pickup_lat', 'pickup_lon']].shape)
print('Transformed shape of PC1: ', pc1.shape)

As we can see, we have reduced from 2 dimensions (2 features) to 1 dimension and this one dimension (or Principal Component), explains 88% of the variance in the original two dimensions. 

In [ ]:
ax = sns.distplot(pc1)
ax.set_xlabel('Principal Component 1');

Below, you can see that our (latitude, longitude) locations have been collapsed to a single line.

In [ ]:
X_new = pca.inverse_transform(pc1)
plt.figure(figsize=(5, 5))
plt.scatter(train['pickup_lon'], 
            train['pickup_lat'], s=0.1, alpha=0.2, label='Pickup Locations')
plt.scatter(X_new[:, 0], X_new[:, 1], s=1, alpha=0.8, label='Pickup Locations\nTransformed to PC1')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pickup locations');

Next, we need to categorizes each pick-up location as 0, 1, or 2 based on the value of each point's first principal component `pc1`.

In [ ]:
out, bins = pd.qcut(pc1.flatten(), 3, labels=[1, 2, 3], retbins=True)
print(out)
print(bins)

Let's see what these regions look like.

In [ ]:
X_new = pca.inverse_transform(pc1)
plt.figure(figsize=(5, 5))
sns.scatterplot(train['pickup_lon'], 
                train['pickup_lat'], 
                hue=out, s=1, alpha=0.2, label='Regions')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pickup locations');

Next, let's create a function `add_region` so we can reuse this workflow.

In [ ]:
pca = PCA(n_components=1)
pca.fit(train[['pickup_lon', 'pickup_lat']])
pc1 = pca.transform(train[['pickup_lon', 'pickup_lat']])
out, bins = pd.qcut(pc1.flatten(), 3, labels=[1, 2, 3], retbins=True)

def add_region(df, pca, bins):
    """Add a "region" column to df_temp."""
    df_temp = df.copy()
    
    # Write your code here
    pc1 = pca.transform(df_temp[['pickup_lon', 'pickup_lat']])
    df_temp['region'] = pd.cut(pc1.flatten(), 
                               [-np.inf, bins[1], bins[2], np.inf], 
                               labels=[1, 2, 3])
    
    return df_temp

# Now let's add 'region' to train, val and test
train = add_region(train, pca, bins)
val = add_region(val, pca, bins)
test = add_region(test, pca, bins)

Below, we compare the `region`'s calculated from the training dataset and applied to the validation dataset.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

sns.scatterplot(data=train, x='pickup_lon', y='pickup_lat', hue='region',
                s=1, alpha=0.2, label='Train Regions', ax=ax1)
ax1.set_xlim([-74.02, -73.92])
ax1.set_ylim([40.7, 40.875])
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.legend()

sns.scatterplot(data=val, x='pickup_lon', y='pickup_lat', hue='region',
                s=1, alpha=0.2, label='Val Regions', ax=ax2)
ax2.set_xlim([-74.02, -73.92])
ax2.set_ylim([40.7, 40.875])
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax2.legend();

Pay attention to this important step in the workflow. You'll notice that `pc1` and the `region` bin limits where computed exclusively from the training dataset and then applied to the validation and test datasets. This is important for ensure no data leakage occurs. `#data-leakage`

## Question 3d
Use `sns.distplot` to create an overlaid histogram comparing the distribution of speeds for morning taxi rides (12am-6am) in the three different regions defined above using `train`. Ponder if there is an association between region and average speed during the night.

In [ ]:
# Write your code here
...

Lastly, let's create a feature set that includes our features of interest. Quantitative features are converted to standard units using `StandardScaler`, while categorical features are converted to dummy variables using `pd.get_dummies`. The `period` is not included because it is a linear combination of the `hour`. The `weekend` variable is not included because it is a linear combination of the `day`.  The `speed` is **NOT** included because it was computed from the `duration` and the `distance`. This would be an example of a type of data leakage called target leakage. This occurs when information about the training target leaks into the features.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define numerical features to use for modelling
num_features = ['pickup_lon', 'pickup_lat', 'dropoff_lon', 'dropoff_lat', 
                'distance']

# Define categorical features to use for modelling
cat_features = ['hour', 'day', 'region']

# Fit scaler (basically get the mean and stdev) for the training data
scaler = StandardScaler()
scaler.fit(train[num_features])

def create_features(df):
    """Create a feature set from taxi ride dataframe df."""
    scaled = df[num_features].copy()
    
    # Convert numeric features to standard units
    scaled.iloc[:, :] = scaler.transform(scaled) 
    
    # Convert categorical features using dummy encoding
    categoricals = [pd.get_dummies(df[s], prefix=s, drop_first=True) for s in cat_features]
    
    return pd.concat([scaled] + categoricals, axis=1)

# Let's test our function
create_features(train).head()

# 4. Model Selection
In this part, you will select a regression model to predict the duration of a taxi ride.

## Question 4a
Assign `constant_rmse` to the root mean squared error on the validation set for a constant model that always predicts the mean duration of all training set taxi rides. Its always benefitial to have a simple (naive) baseline to compare our more sophisticated models too.

In [ ]:
def rmse(errors):
    """Return the root mean squared error."""
    return np.sqrt(np.mean(errors ** 2))

# Write your code here
constant_rmse = ...

# Print score
print('Constant model validation RMSE: {} seconds'.format(constant_rmse))

## Question 4b
Assign `simple_rmse` to the root mean squared error on the validation set for a simple linear regression model that uses only the distance of the taxi ride as a feature (and includes an intercept).

Simple linear regression means that there is only one feature. Multiple linear regression means that there is more than one feature. In either case, you can use the `LinearRegression` model from `sklearn` to fit the parameters to data.

In [ ]:
from sklearn.linear_model import LinearRegression

# Write your code here
...
simple_rmse = ...

# Print score
print('Simple linear regression model validation RMSE: {} seconds'.format(simple_rmse))

## Question 4c
Assign `linear_rmse` to the root mean squared error on the validation set for a linear regression model fitted to the training set without regularization, using the feature set defined by the `create_features` function from Part 3.

In [ ]:
# Write your code here
...
linear_rmse = ...

# Print score
print('Multiple linear regression model validation RMSE: {} seconds'.format(linear_rmse))

## Question 4d
For each possible value of `period`, fit an unregularized linear regression model to the subset of the training set in that `period`.  Assign `period_rmse` to the root mean squared error on the validation set for a model that first chooses linear regression parameters based on the observed `period` of the taxi ride, then predicts the duration using those parameters. Again, fit to the training set and use the `create_features` function for features.

In [ ]:
# Write your code here
model = LinearRegression(fit_intercept=True)
val_errors = []

for period in np.unique(train['period']):
    
    # Filter to period
    period_train = ...
    period_val = ...
    
    # Fit model
    model.fit(...)
    
    # Compute period errors
    period_errors = ...

    # Collect errors
    val_errors.extend(period_errors)

# Compute val score
period_rmse = rmse(np.array(val_errors))

# Print score
print('Period linear regression model validation RMSE: {} seconds'.format(period_rmse))

This approach is a simple form of decision tree regression, where a different regression function is estimated for each possible choice among a collection of choices. In this case, the depth of the tree is only 1.

## Question 4e
Instead of predicting duration directly, an alternative is to predict the average speed of the taxi ride using linear regression, then compute an estimate of the duration from the predicted speed and observed distance for each ride.

Assign `speed_rmse` to the root mean squared error in the duration predicted by a model that first predicts speed as a linear combination of features from the `create_features` function, fitted on the training set, then calculates duration from the predicted speed and observed distance for the validation set.

Hint: Speed is in miles per hour, but duration is measured in seconds. You'll need the fact that there are 60 * 60 = 3,600 seconds in an hour.

In [ ]:
# Write your code here
...
speed_rmse = ...

# Print score
print('Speed multiple linear regression model validation RMSE: {} seconds'.format(speed_rmse))

At this point, think about why predicting speed leads to a more accurate regression model than predicting duration directly. Consider the figure below.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

sns.distplot(train['duration'], ax=ax1)
ax1.set_xlabel('Duration, seconds')

sns.distplot(train['speed'], ax=ax2)
ax2.set_xlabel('Speed, mph')

The `duration` has a large right-skew and a larger dynamics range, which can create challenges. `speed` is much closed to a symmetric distribution, which is the likely explanation for this improved performance. Another option to try would be to use a `log` transformation on `duration`.

In [ ]:
ax= sns.distplot(np.log(train['duration']))
ax.set_xlabel('log Duration, seconds')

Next, let's select the best model based on its validation score.

In [ ]:
models = ['constant', 'simple', 'linear', 'period', 'speed']
pd.DataFrame.from_dict({
    'Model': models,
    'Val RMSE': [eval(m + '_rmse') for m in models]
}).set_index('Model').plot(kind='barh');

Based on the results presented above, we select the `speed` model, which has the lowest RMSE on the validation dataset.

## Question 4f
The last step is to unlock out test dataset and compute the RMSE for the `speed` model as our final evaluation of the model's generalization error.

In [ ]:
# Write your code here
speed_rmse_test = ...

# Print score
print('Speed multiple linear regression model test RMSE: {} seconds'.format(speed_rmse_test))

**Congratulation, you're done Assignment 7. Review your answers and clean up that code before submitting on Quercus. `#cleancode`**